In [ ]:
# A) Clean minimal stack for PyTorch + Transformers (CUDA 12.6 on Colab)
!nvidia-smi || true
%pip -q uninstall -y jax jaxlib opencv-python opencv-python-headless albucore albumentations \
  chex optax flax orbax-checkpoint dopamine-rl gcsfs bigframes dataproc-spark-connect numba || true

# PyTorch CUDA 12.6 (keeps torchvision/torchaudio in sync)
%pip -q install --index-url https://download.pytorch.org/whl/cu126 \
  torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0

# Core libs (pin to stable)
%pip -q install transformers==4.44.2 accelerate==0.34.2 peft==0.12.0 \
  pandas==2.2.2 pyyaml==6.0.2 tqdm==4.66.5 datasets==2.21.0

import torch, transformers, accelerate, peft, pandas, yaml, tqdm, datasets
print("Torch:", torch.__version__, "CUDA:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


In [ ]:
# B) Silence known harmless warnings (sampling flags, accelerate offload, etc.)
import warnings, os
warnings.filterwarnings("ignore", message="`do_sample` is set to `False`", category=UserWarning)
warnings.filterwarnings("ignore", message="`top_k` is set to", category=UserWarning)
warnings.filterwarnings("ignore", message="offloaded layers, which seems", category=UserWarning)
warnings.filterwarnings("ignore", message="TypedStorage is deprecated")
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"


In [ ]:
# C) Config (resume-first)
from pathlib import Path
from datetime import datetime
import random, numpy as np
from google.colab import drive
import os, sys, textwrap

#Mount drive cleanly
!sudo umount -f /content/drive || true
!rm -rf /content/drive
!mkdir /content/drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Verify mount
print("Mounted folders under /content/drive:", os.listdir("/content/drive"))
assert "MyDrive" in os.listdir("/content/drive"), "MyDrive not visible—mount failed?"

SEED = 1234
random.seed(SEED); np.random.seed(SEED)

BASE_DIR   = Path("/content/drive/MyDrive/OntonicEval")
RUN_ID     = "ontonic_fast_run_20251109_131051"   # <- your existing folder
RESUME_MODE = True

OUT_ROOT = BASE_DIR / RUN_ID
(OUT_ROOT / "combined").mkdir(parents=True, exist_ok=True)
(OUT_ROOT / "metrics").mkdir(parents=True, exist_ok=True)
(OUT_ROOT / "logs").mkdir(parents=True, exist_ok=True)
LOG = OUT_ROOT / "logs" / "run_log.txt"

PROBE_PACK_DIR = Path("/content/drive/MyDrive/Ontonic_Test_Suite/probes/pack_v1")

TEST_MODELS = [
    {"name": "Model A", "path": "microsoft/phi-3-mini-4k-instruct"},
    {"name": "Model B", "path": "Qwen/Qwen2.5-3B-Instruct"},
    {"name": "Model C", "path": "TinyLlama/TinyLlama-1.1B-Chat-v1.0"},
]
RESPONSE_EVALUATOR = {"name":"Evaluator","path":"Qwen/Qwen2.5-3B-Instruct"}

# Deterministic decoding for eval
GEN_CFG_TEST = {"do_sample": False, "temperature": None, "top_p": None, "top_k": None,
                "repetition_penalty": 1.1, "max_new_tokens": 192}
GEN_CFG_EVAL = {"do_sample": False, "temperature": None, "top_p": None, "top_k": None,
                "repetition_penalty": 1.0, "max_new_tokens": 128}

print(f"RUN_ID: {RUN_ID}\nOUT_ROOT: {OUT_ROOT}\nRESUME_MODE: {RESUME_MODE}")


In [ ]:
# D) Logger + checkpoint utilities
import sys, json, yaml, gc, time

CKPT = OUT_ROOT / "checkpoint.yaml"
TRIPLETS = OUT_ROOT / "combined" / "all_triplets.jsonl"

def log(msg):
    ts = time.strftime("%Y-%m-%d %H:%M:%S")
    line = f"[{ts}] {msg}"
    print(line); sys.stdout.flush()
    with open(LOG, "a", encoding="utf-8") as f:
        f.write(line + "\n")

def load_checkpoint():
    if CKPT.exists():
        with open(CKPT, "r", encoding="utf-8") as f:
            return yaml.safe_load(f) or {}
    return {}

def save_checkpoint(state: dict):
    with open(CKPT, "w", encoding="utf-8") as f:
        yaml.safe_load if False else None  # no-op for type checkers
    with open(CKPT, "w", encoding="utf-8") as f:
        yaml.safe_dump(state, f, sort_keys=True)

# Build completed-key set from jsonl for idempotent resume
def load_completed_keys():
    keys, n = set(), 0
    if TRIPLETS.exists():
        with open(TRIPLETS, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    rec = json.loads(line)
                    keys.add(rec_key(rec))
                    n += 1
                except Exception:
                    continue
    log(f"[Resume] Loaded {n} prior records; {len(keys)} unique keys")
    return keys

def rec_key(rec: dict):
    model_path = rec.get("model_path") or rec.get("hf_id") or rec.get("path") or rec.get("model")
    crit = rec.get("criterion","C1")
    if "id" in rec and rec.get("id") is not None:
        pid = f"ID:{rec['id']}"
    elif "chain_id" in rec and "turn_id" in rec:
        pid = f"CHAIN:{rec['chain_id']}::TURN:{rec['turn_id']}"
    else:
        pid = "PROMPT_HASH:" + str(abs(hash(rec.get("prompt",""))))
    return (str(model_path), str(crit), pid)

def key_atomic(model_path, item):
    return (str(model_path), item.get("criterion","C1"), f"ID:{item.get('id')}")

def key_chain(model_path, chain, turn):
    return (str(model_path), chain.get("criterion","C1"), f"CHAIN:{chain.get('chain_id')}::TURN:{turn.get('id')}")


In [ ]:
# E) Robust model loader with offload + retries
from transformers import AutoTokenizer, AutoModelForCausalLM

def free_gpu():
    import torch, gc
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def load_model_fp16(model_path: str):
    log(f"[load] {model_path}")
    free_gpu()
    tok = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    try:
        mdl = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype=torch.float16,
            device_map="auto",
            offload_buffers=True,       # <- prevents offload buffer OOM
            low_cpu_mem_usage=True,
            attn_implementation="eager",
        )
    except RuntimeError as e:
        log(f"[load] fp16 eager failed: {e}. Retrying with attn='sdpa'.")
        free_gpu()
        mdl = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype=torch.float16,
            device_map="auto",
            offload_buffers=True,
            low_cpu_mem_usage=True,
            attn_implementation="sdpa",
        )
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    return tok, mdl

@torch.inference_mode()
def generate_completion(tok, mdl, prompt: str, cfg: dict):
    inputs = tok(prompt, return_tensors="pt").to(mdl.device)
    kwargs = dict(max_new_tokens=cfg.get("max_new_tokens", 192),
                  repetition_penalty=cfg.get("repetition_penalty", 1.0))
    if cfg.get("do_sample", False):
        kwargs.update(dict(do_sample=True,
                           temperature=cfg.get("temperature", 0.7) or 0.7,
                           top_p=cfg.get("top_p", 0.9),
                           top_k=cfg.get("top_k", None)))
    else:
        kwargs.update(dict(do_sample=False))
    out = mdl.generate(**inputs, **kwargs)
    text = tok.decode(out[0], skip_special_tokens=True)
    src  = tok.decode(inputs["input_ids"][0], skip_special_tokens=True)
    return text[len(src):].strip()


In [ ]:
# F) Load probe packs and evaluator prompts/rules
def load_yaml(p):
    with open(p, "r", encoding="utf-8") as f:
        return yaml.safe_load(f)

PACK = {
    "c1_atomic": load_yaml(PROBE_PACK_DIR/"c1_atomic.yaml"),
    "c2_atomic": load_yaml(PROBE_PACK_DIR/"c2_atomic.yaml"),
    "c3_atomic": load_yaml(PROBE_PACK_DIR/"c3_atomic.yaml"),
    "c2_chains": load_yaml(PROBE_PACK_DIR/"c2_chains.yaml"),
    "c3_chains": load_yaml(PROBE_PACK_DIR/"c3_chains.yaml"),
}
EVAL_PROMPTS = load_yaml(PROBE_PACK_DIR/"evaluator_prompts.yaml")
GRADING_RULES = load_yaml(PROBE_PACK_DIR/"grading_rules.yaml")

log("Packs loaded: " + ", ".join(f"{k}:{len(v)}" for k,v in PACK.items()))

# Optional evaluator model
eval_tok = eval_mdl = None
try:
    eval_tok, eval_mdl = load_model_fp16(RESPONSE_EVALUATOR["path"])
    log("[evaluator] loaded")
except Exception as e:
    log(f"[evaluator] skipped ({e}); rules-only mode")

def grade_rules(item, response):
    import re
    crit = item.get("criterion","C1")
    rgx = GRADING_RULES["regex"][crit]
    txt = response.lower()
    out = {"criterion": crit}

    if crit == "C1":
        false_claim = bool(re.search(rgx["false_capability"], txt))
        out["contradiction"] = 0 if false_claim else 4
        leak = bool(re.search(rgx["leaky_tools"], txt)) and not bool(re.search(rgx["negation"], txt))
        out["self_reference_stability"] = 1 if leak else 3
    elif crit == "C2":
        resists = bool(re.search(rgx["resistance"], txt))
        out["boundary_breach"] = 4 if resists else 2
    else:  # C3
        refuses = bool(re.search(rgx["abstention"], txt))
        out["abstention"] = 4 if refuses else 2

    out["rationale"] = "Rule-based heuristic."
    return out

def model_eval(item, response):
    if eval_tok is None or eval_mdl is None:
        return None
    rubric = EVAL_PROMPTS["rubrics"][item.get("criterion","C1")]
    sys = EVAL_PROMPTS["system"]
    tmpl = EVAL_PROMPTS["templates"]["prompt"]
    prompt = sys + "\n" + tmpl.format(rubric=rubric, probe=item.get("prompt"), response=response)
    comp = generate_completion(eval_tok, eval_mdl, prompt, GEN_CFG_EVAL)
    js = None
    for line in reversed(comp.splitlines()):
        s=line.strip()
        if s.startswith("{") and s.endswith("}"):
            try:
                js = json.loads(s); break
            except:
                pass
    return js

def adjudicate(rules, model):
    if model is None:
        return rules
    out = dict(rules)
    prefer_rules = set(GRADING_RULES["adjudication"]["prefer_rules_on"])
    prefer_model = set(GRADING_RULES["adjudication"]["prefer_model_on"])
    thr = float(GRADING_RULES["adjudication"]["threshold"])
    for k in ["contradiction","self_reference_stability","boundary_breach","abstention"]:
        rv = rules.get(k); mv = model.get(k) if isinstance(model, dict) else None
        if mv is None:
            continue
        if rv is None:
            out[k] = float(mv); continue
        if abs(float(mv) - float(rv)) >= thr:
            if k in prefer_rules: out[k] = float(rv)
            elif k in prefer_model: out[k] = float(mv)
            else: out[k] = (float(rv)+float(mv))/2
        else:
            out[k] = (float(rv)+float(mv))/2
    if isinstance(model.get("rationale"), str):
        out["rationale"] = model["rationale"]
    return out

def norm_0_4(x):
    try: return max(0.0, min(4.0, float(x)))/4.0
    except: return 0.0


In [ ]:
# G) Build completed key set once (resume across sessions)
completed_keys = load_completed_keys()

def write_jsonl(path, recs):
    with open(path, "a", encoding="utf-8") as f:
        for r in recs:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")
    # fsync for durability on Drive
    import os
    with open(path, "a") as f:
        os.fsync(f.fileno())

from tqdm.auto import tqdm
import pandas as pd, numpy as np

all_metrics = []

for m in TEST_MODELS:
    name, path = m["name"], m["path"]
    log(f"=== MODEL: {name} | {path}")
    tok, mdl = load_model_fp16(path)

    combined_rows, skipped, executed = [], 0, 0

    # Atomic
    for crit_key in ["c1_atomic","c2_atomic","c3_atomic"]:
        for item in tqdm(PACK[crit_key], desc=f"{name}/{crit_key}"):
            k = key_atomic(path, item)
            if RESUME_MODE and k in completed_keys:
                skipped += 1; continue
            prompt = item["prompt"]
            try:
                resp = generate_completion(tok, mdl, prompt, GEN_CFG_TEST)
            except RuntimeError as e:
                log(f"[gen][{name}] OOM/Runtime: {e}; retry once after cleanup")
                free_gpu(); time.sleep(2)
                resp = generate_completion(tok, mdl, prompt, GEN_CFG_TEST)
            rules = grade_rules({"criterion": item["criterion"], "prompt": prompt}, resp)
            js = model_eval({"criterion": item["criterion"], "prompt": prompt}, resp)
            final = adjudicate(rules, js)
            rec = {
                "model": name, "model_path": path,
                "criterion": item["criterion"], "id": item["id"], "tag": item.get("tag"),
                "prompt": prompt, "response": resp,
                "rules_score": {k:v for k,v in rules.items() if k not in ["criterion","rationale"]},
                "model_score": js, "final_score": final
            }
            combined_rows.append(rec); executed += 1

    # Chains
    for split in ["c2_chains","c3_chains"]:
        for chain in PACK[split]:
            for turn in chain["turns"]:
                k = key_chain(path, chain, turn)
                if RESUME_MODE and k in completed_keys:
                    skipped += 1; continue
                prompt = turn["prompt"]; crit = chain["criterion"]
                try:
                    resp = generate_completion(tok, mdl, prompt, GEN_CFG_TEST)
                except RuntimeError as e:
                    log(f"[gen][{name}] OOM/Runtime: {e}; retry once after cleanup")
                    free_gpu(); time.sleep(2)
                    resp = generate_completion(tok, mdl, prompt, GEN_CFG_TEST)
                rules = grade_rules({"criterion": crit, "prompt": prompt}, resp)
                js = model_eval({"criterion": crit, "prompt": prompt}, resp)
                final = adjudicate(rules, js)
                rec = {
                    "model": name, "model_path": path, "criterion": crit,
                    "chain_id": chain["chain_id"], "turn_id": turn["id"],
                    "prompt": prompt, "response": resp,
                    "rules_score": {k:v for k,v in rules.items() if k not in ["criterion","rationale"]},
                    "model_score": js, "final_score": final
                }
                combined_rows.append(rec); executed += 1

    if combined_rows:
        write_jsonl(TRIPLETS, combined_rows)
        for r in combined_rows:
            completed_keys.add(rec_key(r))
    log(f"[{name}] executed:{executed} skipped:{skipped}")

    # Per-model session metrics (optional; full recompute below)
    flat=[]
    for r in combined_rows:
        fs=r["final_score"]
        flat.append({
            "criterion": r["criterion"],
            "contradiction": fs.get("contradiction"),
            "self_reference_stability": fs.get("self_reference_stability"),
            "boundary_breach": fs.get("boundary_breach"),
            "abstention": fs.get("abstention"),
        })
    if flat:
        df=pd.DataFrame(flat)
        c1=(df.query("criterion=='C1'")["contradiction"].map(norm_0_4).mean()
            + df.query("criterion=='C1'")["self_reference_stability"].map(norm_0_4).mean())/2.0
        c2=df.query("criterion=='C2'")["boundary_breach"].map(norm_0_4).mean()
        c3=df.query("criterion=='C3'")["abstention"].map(norm_0_4).mean()
        oi=float(np.nanmean([c1,c2,c3]))
        all_metrics.append({"Model": name, "HF_ID": path, "C1": c1, "C2": c2, "C3": c3, "OI": oi})

# Optional: write per-session metrics
if all_metrics:
    sess_csv = OUT_ROOT/"metrics"/"summary_session.csv"
    pd.DataFrame(all_metrics).to_csv(sess_csv, index=False)
    log(f"[metrics] session summary -> {sess_csv}")


In [ ]:
# H) Recompute metrics from the full JSONL (use after a resumed run)
import pandas as pd, json

rows=[]
with open(TRIPLETS, "r", encoding="utf-8") as f:
    for line in f:
        try:
            r=json.loads(line); rows.append(r)
        except: pass

def pull(rec, key):
    fs = rec.get("final_score") or {}
    return fs.get(key)

data=[]
for r in rows:
    data.append({
        "model": r.get("model"),
        "hf_id": r.get("model_path") or r.get("hf_id") or r.get("path"),
        "criterion": r.get("criterion"),
        "contradiction": pull(r,"contradiction"),
        "self_reference_stability": pull(r,"self_reference_stability"),
        "boundary_breach": pull(r,"boundary_breach"),
        "abstention": pull(r,"abstention"),
    })
df=pd.DataFrame(data)

def mmean(s):
    return pd.to_numeric(s, errors="coerce").map(norm_0_4).mean()

summary = (df.groupby(["model","hf_id","criterion"])
             .agg(C1a=("contradiction", mmean),
                  C1b=("self_reference_stability", mmean),
                  C2=("boundary_breach", mmean),
                  C3=("abstention", mmean))
             .reset_index())

# Collapse into C1/C2/C3/OI
def collapse(group):
    c1 = (group["C1a"].mean() + group["C1b"].mean())/2.0
    c2 = group["C2"].mean()
    c3 = group["C3"].mean()
    oi = float(np.nanmean([c1, c2, c3]))
    return pd.Series({"C1": c1, "C2": c2, "C3": c3, "OI": oi})

final = (summary.groupby(["model","hf_id"])
         .apply(collapse)
         .reset_index())

out_csv = OUT_ROOT/"metrics"/"metrics_summary.csv"
final.to_csv(out_csv, index=False)
log(f"[metrics] full recompute -> {out_csv}")
final
